# TimeGrad: Автореgrессивная диффузия для криптовалют

TimeGrad - это авторегрессивная модель диффузии, разработанная Amazon (2021), которая комбинирует RNN для кодирования контекста с диффузионной моделью для генерации вероятностных прогнозов.

## Основные компоненты TimeGrad:

1. **RNN Encoder**: Кодирует исторический контекст в скрытое состояние
2. **Diffusion Decoder**: Генерирует прогнозы через обратный диффузионный процесс
3. **Autoregressive Generation**: Прогнозы генерируются пошагово

## Преимущества TimeGrad:

- Вероятностные прогнозы с квантилями
- Хорошо работает с мультивариантными временными рядами
- Естественная оценка неопределённости

![TimeGrad Architecture](https://raw.githubusercontent.com/zalandoresearch/pytorch-ts/master/docs/source/_static/timegrad.png)

In [ ]:
# Установка зависимостей
!pip install torch numpy pandas matplotlib seaborn scikit-learn tqdm requests

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from typing import Tuple, Optional, List, Dict
from dataclasses import dataclass
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# Настройка стиля
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('husl')

# Устройство
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Устройство: {device}")

## 1. Получение данных криптовалют

In [ ]:
import requests
from datetime import datetime, timedelta

class CryptoDataFetcher:
    """Получение данных криптовалют через Bybit API."""
    
    BASE_URL = "https://api.bybit.com/v5/market/kline"
    
    def __init__(self):
        self.session = requests.Session()
    
    def fetch_ohlcv(
        self, 
        symbol: str = "BTCUSDT",
        interval: str = "60",  # 1 час
        days: int = 90
    ) -> pd.DataFrame:
        """Получить OHLCV данные."""
        
        all_data = []
        end_time = int(datetime.now().timestamp() * 1000)
        start_time = int((datetime.now() - timedelta(days=days)).timestamp() * 1000)
        
        current_end = end_time
        
        while current_end > start_time:
            params = {
                "category": "spot",
                "symbol": symbol,
                "interval": interval,
                "end": current_end,
                "limit": 200
            }
            
            try:
                response = self.session.get(self.BASE_URL, params=params)
                data = response.json()
                
                if data.get("retCode") != 0:
                    print(f"API Error: {data.get('retMsg')}")
                    break
                
                klines = data.get("result", {}).get("list", [])
                if not klines:
                    break
                
                all_data.extend(klines)
                current_end = int(klines[-1][0]) - 1
                
            except Exception as e:
                print(f"Error: {e}")
                break
        
        if not all_data:
            # Создаём синтетические данные если API недоступен
            return self._generate_synthetic_data(days * 24)
        
        # Преобразуем в DataFrame
        df = pd.DataFrame(all_data, columns=[
            'timestamp', 'open', 'high', 'low', 'close', 'volume', 'turnover'
        ])
        
        df['timestamp'] = pd.to_datetime(df['timestamp'].astype(int), unit='ms')
        for col in ['open', 'high', 'low', 'close', 'volume']:
            df[col] = df[col].astype(float)
        
        df = df.sort_values('timestamp').reset_index(drop=True)
        return df
    
    def _generate_synthetic_data(self, n_points: int) -> pd.DataFrame:
        """Генерация синтетических данных для тестирования."""
        np.random.seed(42)
        
        # Геометрическое броуновское движение
        dt = 1/24  # 1 час
        mu = 0.0001  # drift
        sigma = 0.02  # volatility
        
        S0 = 50000  # начальная цена BTC
        returns = np.random.normal(mu * dt, sigma * np.sqrt(dt), n_points)
        prices = S0 * np.exp(np.cumsum(returns))
        
        # OHLCV
        high = prices * (1 + np.abs(np.random.normal(0, 0.005, n_points)))
        low = prices * (1 - np.abs(np.random.normal(0, 0.005, n_points)))
        open_prices = np.roll(prices, 1)
        open_prices[0] = S0
        volume = np.random.exponential(1000, n_points)
        
        timestamps = pd.date_range(
            end=datetime.now(),
            periods=n_points,
            freq='H'
        )
        
        return pd.DataFrame({
            'timestamp': timestamps,
            'open': open_prices,
            'high': high,
            'low': low,
            'close': prices,
            'volume': volume
        })

# Получаем данные
fetcher = CryptoDataFetcher()
df = fetcher.fetch_ohlcv(symbol="BTCUSDT", interval="60", days=90)
print(f"Получено {len(df)} записей")
print(f"Период: {df['timestamp'].min()} - {df['timestamp'].max()}")
df.head()

In [ ]:
# Визуализация данных
fig, axes = plt.subplots(2, 1, figsize=(14, 8))

# Цена
axes[0].plot(df['timestamp'], df['close'], linewidth=0.8)
axes[0].set_title('BTC/USDT Price (Hourly)', fontsize=14)
axes[0].set_ylabel('Price (USD)')
axes[0].grid(True, alpha=0.3)

# Объём
axes[1].bar(df['timestamp'], df['volume'], width=0.03, alpha=0.7)
axes[1].set_title('Trading Volume', fontsize=14)
axes[1].set_ylabel('Volume')
axes[1].set_xlabel('Date')

plt.tight_layout()
plt.show()

## 2. Feature Engineering

In [ ]:
class FeatureEngineer:
    """Вычисление технических индикаторов."""
    
    @staticmethod
    def compute_returns(df: pd.DataFrame, periods: List[int] = [1, 6, 24]) -> pd.DataFrame:
        """Вычислить логарифмические доходности."""
        for p in periods:
            df[f'return_{p}h'] = np.log(df['close'] / df['close'].shift(p))
        return df
    
    @staticmethod
    def compute_volatility(df: pd.DataFrame, windows: List[int] = [12, 24, 72]) -> pd.DataFrame:
        """Вычислить реализованную волатильность."""
        for w in windows:
            df[f'volatility_{w}h'] = df['return_1h'].rolling(w).std() * np.sqrt(24)
        return df
    
    @staticmethod
    def compute_rsi(df: pd.DataFrame, period: int = 14) -> pd.DataFrame:
        """Вычислить RSI."""
        delta = df['close'].diff()
        gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
        rs = gain / loss
        df['rsi'] = 100 - (100 / (1 + rs))
        return df
    
    @staticmethod
    def compute_macd(df: pd.DataFrame) -> pd.DataFrame:
        """Вычислить MACD."""
        ema12 = df['close'].ewm(span=12, adjust=False).mean()
        ema26 = df['close'].ewm(span=26, adjust=False).mean()
        df['macd'] = ema12 - ema26
        df['macd_signal'] = df['macd'].ewm(span=9, adjust=False).mean()
        df['macd_hist'] = df['macd'] - df['macd_signal']
        return df
    
    @staticmethod
    def compute_bollinger(df: pd.DataFrame, period: int = 20) -> pd.DataFrame:
        """Вычислить Bollinger Bands."""
        sma = df['close'].rolling(period).mean()
        std = df['close'].rolling(period).std()
        df['bb_upper'] = sma + 2 * std
        df['bb_lower'] = sma - 2 * std
        df['bb_width'] = (df['bb_upper'] - df['bb_lower']) / sma
        df['bb_position'] = (df['close'] - df['bb_lower']) / (df['bb_upper'] - df['bb_lower'])
        return df
    
    @staticmethod
    def compute_atr(df: pd.DataFrame, period: int = 14) -> pd.DataFrame:
        """Вычислить Average True Range."""
        high_low = df['high'] - df['low']
        high_close = np.abs(df['high'] - df['close'].shift())
        low_close = np.abs(df['low'] - df['close'].shift())
        tr = pd.concat([high_low, high_close, low_close], axis=1).max(axis=1)
        df['atr'] = tr.rolling(period).mean()
        df['atr_pct'] = df['atr'] / df['close'] * 100
        return df
    
    def compute_all(self, df: pd.DataFrame) -> pd.DataFrame:
        """Вычислить все признаки."""
        df = df.copy()
        df = self.compute_returns(df)
        df = self.compute_volatility(df)
        df = self.compute_rsi(df)
        df = self.compute_macd(df)
        df = self.compute_bollinger(df)
        df = self.compute_atr(df)
        return df.dropna()

# Вычисляем признаки
engineer = FeatureEngineer()
df_features = engineer.compute_all(df)
print(f"Признаки: {list(df_features.columns)}")
print(f"Записей после feature engineering: {len(df_features)}")

## 3. Подготовка данных для TimeGrad

In [ ]:
# Выбираем признаки для модели
feature_columns = [
    'close', 'volume', 
    'return_1h', 'return_6h', 'return_24h',
    'volatility_12h', 'volatility_24h',
    'rsi', 'macd_hist', 'bb_position', 'atr_pct'
]

# Нормализация
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
df_scaled = pd.DataFrame(
    scaler.fit_transform(df_features[feature_columns]),
    columns=feature_columns,
    index=df_features.index
)

print(f"Нормализованные данные: {df_scaled.shape}")
df_scaled.describe()

In [ ]:
@dataclass
class TimeGradConfig:
    """Конфигурация TimeGrad."""
    context_length: int = 72      # 3 дня истории
    prediction_length: int = 24   # 24 часа прогноза
    input_dim: int = 11           # количество признаков
    hidden_dim: int = 128         # размерность RNN
    num_layers: int = 2           # количество слоёв RNN
    diffusion_steps: int = 100    # шаги диффузии
    beta_start: float = 1e-4
    beta_end: float = 0.02
    
config = TimeGradConfig(input_dim=len(feature_columns))
print(f"Config: {config}")

In [ ]:
def create_sequences(
    data: np.ndarray,
    context_length: int,
    prediction_length: int
) -> Tuple[np.ndarray, np.ndarray]:
    """Создать последовательности для обучения."""
    
    total_length = context_length + prediction_length
    n_samples = len(data) - total_length + 1
    
    X = []  # context
    y = []  # target
    
    for i in range(n_samples):
        X.append(data[i:i+context_length])
        y.append(data[i+context_length:i+total_length, 0])  # только close
    
    return np.array(X), np.array(y)

# Создаём последовательности
data_np = df_scaled.values
X, y = create_sequences(data_np, config.context_length, config.prediction_length)

# Train/val/test split
n = len(X)
train_size = int(0.7 * n)
val_size = int(0.15 * n)

X_train, y_train = X[:train_size], y[:train_size]
X_val, y_val = X[train_size:train_size+val_size], y[train_size:train_size+val_size]
X_test, y_test = X[train_size+val_size:], y[train_size+val_size:]

print(f"Train: {X_train.shape}, Val: {X_val.shape}, Test: {X_test.shape}")

In [ ]:
# PyTorch DataLoader
from torch.utils.data import Dataset, DataLoader

class TimeSeriesDataset(Dataset):
    def __init__(self, X: np.ndarray, y: np.ndarray):
        self.X = torch.FloatTensor(X)
        self.y = torch.FloatTensor(y)
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

train_dataset = TimeSeriesDataset(X_train, y_train)
val_dataset = TimeSeriesDataset(X_val, y_val)
test_dataset = TimeSeriesDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

print(f"Train batches: {len(train_loader)}")

## 4. TimeGrad Architecture

TimeGrad состоит из:
1. **RNN Encoder** - кодирует исторический контекст
2. **Diffusion Network** - денойзер для диффузии
3. **Autoregressive Decoder** - генерирует прогнозы пошагово

In [ ]:
class SinusoidalPositionEmbedding(nn.Module):
    """Синусоидальное позиционное кодирование для времени диффузии."""
    
    def __init__(self, dim: int):
        super().__init__()
        self.dim = dim
    
    def forward(self, t: torch.Tensor) -> torch.Tensor:
        device = t.device
        half_dim = self.dim // 2
        embeddings = np.log(10000) / (half_dim - 1)
        embeddings = torch.exp(torch.arange(half_dim, device=device) * -embeddings)
        embeddings = t[:, None] * embeddings[None, :]
        embeddings = torch.cat([embeddings.sin(), embeddings.cos()], dim=-1)
        return embeddings

In [ ]:
class RNNEncoder(nn.Module):
    """RNN энкодер для исторического контекста."""
    
    def __init__(
        self,
        input_dim: int,
        hidden_dim: int,
        num_layers: int = 2,
        dropout: float = 0.1
    ):
        super().__init__()
        
        self.rnn = nn.LSTM(
            input_size=input_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0
        )
        
    def forward(self, x: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor]:
        """
        Args:
            x: [batch, seq_len, input_dim]
        Returns:
            outputs: [batch, seq_len, hidden_dim]
            hidden: final hidden state
        """
        outputs, (h_n, c_n) = self.rnn(x)
        # Берём последний скрытый слой
        hidden = h_n[-1]  # [batch, hidden_dim]
        return outputs, hidden

In [ ]:
class DiffusionDenoiser(nn.Module):
    """Денойзер для TimeGrad."""
    
    def __init__(
        self,
        input_dim: int,
        hidden_dim: int,
        context_dim: int,
        time_dim: int = 64
    ):
        super().__init__()
        
        # Кодирование времени диффузии
        self.time_embed = nn.Sequential(
            SinusoidalPositionEmbedding(time_dim),
            nn.Linear(time_dim, hidden_dim),
            nn.SiLU(),
            nn.Linear(hidden_dim, hidden_dim)
        )
        
        # Объединение входа, контекста и времени
        self.input_proj = nn.Linear(input_dim, hidden_dim)
        self.context_proj = nn.Linear(context_dim, hidden_dim)
        
        # MLP для денойзинга
        self.net = nn.Sequential(
            nn.Linear(hidden_dim * 3, hidden_dim * 2),
            nn.SiLU(),
            nn.Linear(hidden_dim * 2, hidden_dim * 2),
            nn.SiLU(),
            nn.Linear(hidden_dim * 2, hidden_dim),
            nn.SiLU(),
            nn.Linear(hidden_dim, input_dim)
        )
    
    def forward(
        self,
        x: torch.Tensor,      # [batch, input_dim] - зашумлённый вход
        t: torch.Tensor,      # [batch] - шаг диффузии
        context: torch.Tensor # [batch, context_dim] - контекст от RNN
    ) -> torch.Tensor:
        """
        Предсказание шума.
        """
        # Кодируем компоненты
        x_emb = self.input_proj(x)
        t_emb = self.time_embed(t)
        ctx_emb = self.context_proj(context)
        
        # Объединяем
        combined = torch.cat([x_emb, t_emb, ctx_emb], dim=-1)
        
        # Предсказываем шум
        noise_pred = self.net(combined)
        
        return noise_pred

In [ ]:
class TimeGrad(nn.Module):
    """TimeGrad: Autoregressive Denoising Diffusion Model for Time Series."""
    
    def __init__(self, config: TimeGradConfig):
        super().__init__()
        
        self.config = config
        
        # RNN Encoder
        self.encoder = RNNEncoder(
            input_dim=config.input_dim,
            hidden_dim=config.hidden_dim,
            num_layers=config.num_layers
        )
        
        # Diffusion Denoiser (для одномерного выхода - close price)
        self.denoiser = DiffusionDenoiser(
            input_dim=1,  # предсказываем только close
            hidden_dim=config.hidden_dim,
            context_dim=config.hidden_dim
        )
        
        # Noise schedule
        self.register_buffer(
            'betas', 
            torch.linspace(config.beta_start, config.beta_end, config.diffusion_steps)
        )
        alphas = 1 - self.betas
        self.register_buffer('alphas', alphas)
        self.register_buffer('alphas_cumprod', torch.cumprod(alphas, dim=0))
        self.register_buffer('sqrt_alphas_cumprod', torch.sqrt(self.alphas_cumprod))
        self.register_buffer(
            'sqrt_one_minus_alphas_cumprod', 
            torch.sqrt(1 - self.alphas_cumprod)
        )
    
    def forward_diffusion(
        self, 
        x0: torch.Tensor, 
        t: torch.Tensor,
        noise: Optional[torch.Tensor] = None
    ) -> Tuple[torch.Tensor, torch.Tensor]:
        """
        Прямой диффузионный процесс.
        
        Args:
            x0: [batch, dim] - чистые данные
            t: [batch] - шаги времени
        """
        if noise is None:
            noise = torch.randn_like(x0)
        
        sqrt_alpha = self.sqrt_alphas_cumprod[t].unsqueeze(-1)
        sqrt_one_minus_alpha = self.sqrt_one_minus_alphas_cumprod[t].unsqueeze(-1)
        
        xt = sqrt_alpha * x0 + sqrt_one_minus_alpha * noise
        
        return xt, noise
    
    def compute_loss(
        self,
        context: torch.Tensor,  # [batch, context_len, input_dim]
        target: torch.Tensor    # [batch, pred_len]
    ) -> torch.Tensor:
        """
        Вычислить loss для обучения.
        """
        batch_size = context.shape[0]
        device = context.device
        
        # Кодируем контекст
        _, hidden = self.encoder(context)
        
        # Выбираем случайный шаг прогноза для обучения
        pred_step = torch.randint(0, self.config.prediction_length, (batch_size,))
        x0 = target[torch.arange(batch_size), pred_step].unsqueeze(-1)  # [batch, 1]
        
        # Случайный шаг диффузии
        t = torch.randint(0, self.config.diffusion_steps, (batch_size,), device=device)
        
        # Прямая диффузия
        xt, noise = self.forward_diffusion(x0, t)
        
        # Предсказание шума
        noise_pred = self.denoiser(xt, t.float(), hidden)
        
        # MSE loss
        loss = F.mse_loss(noise_pred, noise)
        
        return loss
    
    @torch.no_grad()
    def sample(
        self,
        context: torch.Tensor,
        num_samples: int = 100
    ) -> torch.Tensor:
        """
        Генерировать прогнозы через DDPM sampling.
        
        Args:
            context: [batch, context_len, input_dim]
            num_samples: количество Monte Carlo samples
            
        Returns:
            forecasts: [batch, num_samples, pred_len]
        """
        batch_size = context.shape[0]
        device = context.device
        
        # Кодируем контекст
        _, hidden = self.encoder(context)
        
        all_forecasts = []
        
        for _ in range(num_samples):
            # Авторегрессивная генерация
            forecasts = []
            
            for step in range(self.config.prediction_length):
                # Начинаем с шума
                xt = torch.randn(batch_size, 1, device=device)
                
                # Обратный процесс диффузии
                for t in reversed(range(self.config.diffusion_steps)):
                    t_tensor = torch.full((batch_size,), t, device=device, dtype=torch.float)
                    
                    # Предсказываем шум
                    noise_pred = self.denoiser(xt, t_tensor, hidden)
                    
                    # DDPM update
                    alpha = self.alphas[t]
                    alpha_cumprod = self.alphas_cumprod[t]
                    beta = self.betas[t]
                    
                    if t > 0:
                        noise = torch.randn_like(xt)
                        sigma = torch.sqrt(beta)
                    else:
                        noise = 0
                        sigma = 0
                    
                    xt = (1 / torch.sqrt(alpha)) * (
                        xt - (beta / torch.sqrt(1 - alpha_cumprod)) * noise_pred
                    ) + sigma * noise
                
                forecasts.append(xt)
            
            # [batch, pred_len]
            sample_forecast = torch.cat(forecasts, dim=-1)
            all_forecasts.append(sample_forecast)
        
        # [batch, num_samples, pred_len]
        return torch.stack(all_forecasts, dim=1)

# Создаём модель
model = TimeGrad(config).to(device)
print(f"Параметров: {sum(p.numel() for p in model.parameters()):,}")

## 5. Обучение TimeGrad

In [ ]:
def train_epoch(
    model: TimeGrad,
    loader: DataLoader,
    optimizer: torch.optim.Optimizer,
    device: torch.device
) -> float:
    """Обучить одну эпоху."""
    model.train()
    total_loss = 0
    
    for context, target in loader:
        context = context.to(device)
        target = target.to(device)
        
        optimizer.zero_grad()
        loss = model.compute_loss(context, target)
        loss.backward()
        
        # Gradient clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        
        optimizer.step()
        total_loss += loss.item()
    
    return total_loss / len(loader)


def validate(
    model: TimeGrad,
    loader: DataLoader,
    device: torch.device
) -> float:
    """Валидация модели."""
    model.eval()
    total_loss = 0
    
    with torch.no_grad():
        for context, target in loader:
            context = context.to(device)
            target = target.to(device)
            loss = model.compute_loss(context, target)
            total_loss += loss.item()
    
    return total_loss / len(loader)

In [ ]:
# Обучение
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-5)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=50)

n_epochs = 50
train_losses = []
val_losses = []
best_val_loss = float('inf')

for epoch in tqdm(range(n_epochs), desc="Training"):
    train_loss = train_epoch(model, train_loader, optimizer, device)
    val_loss = validate(model, val_loader, device)
    
    train_losses.append(train_loss)
    val_losses.append(val_loss)
    
    scheduler.step()
    
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model_state = model.state_dict().copy()
    
    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch+1}: Train Loss = {train_loss:.6f}, Val Loss = {val_loss:.6f}")

# Загружаем лучшую модель
model.load_state_dict(best_model_state)
print(f"\nЛучший Val Loss: {best_val_loss:.6f}")

In [ ]:
# График обучения
fig, ax = plt.subplots(figsize=(10, 5))

ax.plot(train_losses, label='Train Loss', alpha=0.8)
ax.plot(val_losses, label='Val Loss', alpha=0.8)
ax.set_xlabel('Epoch')
ax.set_ylabel('Loss')
ax.set_title('TimeGrad Training Progress')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 6. Генерация прогнозов

In [ ]:
def generate_forecasts(
    model: TimeGrad,
    context: torch.Tensor,
    num_samples: int = 100
) -> Dict[str, np.ndarray]:
    """
    Генерировать вероятностные прогнозы.
    
    Returns:
        dict с mean, std, quantiles
    """
    model.eval()
    
    with torch.no_grad():
        # [batch, num_samples, pred_len]
        samples = model.sample(context, num_samples)
    
    samples_np = samples.cpu().numpy()
    
    return {
        'samples': samples_np,
        'mean': samples_np.mean(axis=1),
        'std': samples_np.std(axis=1),
        'q10': np.percentile(samples_np, 10, axis=1),
        'q50': np.percentile(samples_np, 50, axis=1),
        'q90': np.percentile(samples_np, 90, axis=1),
    }

In [ ]:
# Генерируем прогнозы на тестовых данных
test_context, test_target = next(iter(test_loader))
test_context = test_context.to(device)

print("Генерация прогнозов (это может занять время)...")
forecasts = generate_forecasts(model, test_context[:5], num_samples=50)

print(f"Форма прогнозов: {forecasts['mean'].shape}")

In [ ]:
# Визуализация прогнозов
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
axes = axes.flatten()

for i in range(4):
    ax = axes[i]
    
    # Истинные значения (нормализованные)
    true_values = test_target[i].numpy()
    
    # Прогнозы
    mean = forecasts['mean'][i]
    q10 = forecasts['q10'][i]
    q90 = forecasts['q90'][i]
    
    hours = np.arange(1, config.prediction_length + 1)
    
    # Доверительный интервал
    ax.fill_between(hours, q10, q90, alpha=0.3, label='80% CI')
    ax.plot(hours, mean, 'b-', linewidth=2, label='Forecast Mean')
    ax.plot(hours, true_values, 'r--', linewidth=2, label='Actual')
    
    ax.set_xlabel('Hours Ahead')
    ax.set_ylabel('Normalized Price')
    ax.set_title(f'Sample {i+1}')
    ax.legend()
    ax.grid(True, alpha=0.3)

plt.suptitle('TimeGrad Probabilistic Forecasts', fontsize=14)
plt.tight_layout()
plt.show()

## 7. Оценка качества прогнозов

In [ ]:
def compute_metrics(
    predictions: np.ndarray,
    targets: np.ndarray,
    quantile_forecasts: Optional[Dict] = None
) -> Dict[str, float]:
    """Вычислить метрики качества прогнозов."""
    
    # Point forecast metrics
    mse = np.mean((predictions - targets) ** 2)
    rmse = np.sqrt(mse)
    mae = np.mean(np.abs(predictions - targets))
    
    # MAPE (avoiding division by zero)
    mask = targets != 0
    mape = np.mean(np.abs((predictions[mask] - targets[mask]) / targets[mask])) * 100
    
    metrics = {
        'MSE': mse,
        'RMSE': rmse,
        'MAE': mae,
        'MAPE': mape
    }
    
    # Calibration metrics for probabilistic forecasts
    if quantile_forecasts is not None:
        q10 = quantile_forecasts['q10']
        q90 = quantile_forecasts['q90']
        
        # Coverage: какая доля истинных значений попала в интервал
        in_interval = (targets >= q10) & (targets <= q90)
        coverage = np.mean(in_interval)
        
        # Interval width
        interval_width = np.mean(q90 - q10)
        
        metrics['Coverage_80'] = coverage
        metrics['Interval_Width'] = interval_width
        
        # CRPS (Continuous Ranked Probability Score) - simplified
        samples = quantile_forecasts.get('samples')
        if samples is not None:
            # Approximate CRPS
            crps = np.mean(np.abs(samples - targets[:, np.newaxis, :]).mean(axis=1))
            metrics['CRPS'] = crps
    
    return metrics

In [ ]:
# Оценка на тестовых данных
all_predictions = []
all_targets = []
all_forecasts = {'q10': [], 'q90': [], 'samples': []}

print("Оценка на тестовых данных...")
for context, target in tqdm(test_loader, desc="Evaluating"):
    context = context.to(device)
    
    forecasts = generate_forecasts(model, context, num_samples=30)
    
    all_predictions.append(forecasts['mean'])
    all_targets.append(target.numpy())
    all_forecasts['q10'].append(forecasts['q10'])
    all_forecasts['q90'].append(forecasts['q90'])
    all_forecasts['samples'].append(forecasts['samples'])

predictions = np.concatenate(all_predictions, axis=0)
targets = np.concatenate(all_targets, axis=0)

quantile_forecasts = {
    'q10': np.concatenate(all_forecasts['q10'], axis=0),
    'q90': np.concatenate(all_forecasts['q90'], axis=0),
    'samples': np.concatenate(all_forecasts['samples'], axis=0)
}

metrics = compute_metrics(predictions, targets, quantile_forecasts)

print("\n=== Метрики TimeGrad ===")
for name, value in metrics.items():
    print(f"{name}: {value:.4f}")

In [ ]:
# Калибровочный график
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# 1. Предсказания vs Факт
ax1 = axes[0]
ax1.scatter(targets.flatten(), predictions.flatten(), alpha=0.1, s=1)
min_val, max_val = targets.min(), targets.max()
ax1.plot([min_val, max_val], [min_val, max_val], 'r--', label='Perfect')
ax1.set_xlabel('Actual')
ax1.set_ylabel('Predicted')
ax1.set_title('Predictions vs Actual')
ax1.legend()
ax1.grid(True, alpha=0.3)

# 2. Распределение ошибок по горизонту
ax2 = axes[1]
errors_by_horizon = np.abs(predictions - targets)
mae_by_horizon = errors_by_horizon.mean(axis=0)
std_by_horizon = errors_by_horizon.std(axis=0)

hours = np.arange(1, config.prediction_length + 1)
ax2.fill_between(hours, mae_by_horizon - std_by_horizon, 
                  mae_by_horizon + std_by_horizon, alpha=0.3)
ax2.plot(hours, mae_by_horizon, 'b-', linewidth=2)
ax2.set_xlabel('Forecast Horizon (hours)')
ax2.set_ylabel('MAE')
ax2.set_title('Error by Forecast Horizon')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 8. Сравнение с базовыми моделями

In [ ]:
class NaiveForecaster:
    """Наивный прогноз: повторяем последнее значение."""
    
    def predict(self, context: np.ndarray, horizon: int) -> np.ndarray:
        last_value = context[:, -1, 0]  # последний close
        return np.tile(last_value[:, np.newaxis], (1, horizon))


class SeasonalNaiveForecaster:
    """Сезонный наивный: повторяем значения из прошлой недели."""
    
    def predict(self, context: np.ndarray, horizon: int) -> np.ndarray:
        # Берём значения из того же времени неделю назад (если есть)
        seasonal_period = 24 * 7  # неделя в часах
        if context.shape[1] >= seasonal_period:
            return context[:, -seasonal_period:-seasonal_period+horizon, 0]
        else:
            # Fallback на наивный
            return NaiveForecaster().predict(context, horizon)


# Оценка базовых моделей
naive = NaiveForecaster()

naive_predictions = []
for context, target in test_loader:
    pred = naive.predict(context.numpy(), config.prediction_length)
    naive_predictions.append(pred)

naive_predictions = np.concatenate(naive_predictions, axis=0)
naive_metrics = compute_metrics(naive_predictions, targets)

print("=== Сравнение моделей ===")
print(f"\nTimeGrad:")
print(f"  RMSE: {metrics['RMSE']:.4f}")
print(f"  MAE:  {metrics['MAE']:.4f}")
print(f"  Coverage (80%): {metrics.get('Coverage_80', 'N/A'):.2%}")

print(f"\nNaive (Last Value):")
print(f"  RMSE: {naive_metrics['RMSE']:.4f}")
print(f"  MAE:  {naive_metrics['MAE']:.4f}")

print(f"\nОтносительное улучшение RMSE: {(1 - metrics['RMSE']/naive_metrics['RMSE'])*100:.1f}%")

## 9. Генерация реального прогноза

In [ ]:
def generate_real_forecast(
    model: TimeGrad,
    df: pd.DataFrame,
    feature_columns: List[str],
    scaler: StandardScaler,
    config: TimeGradConfig,
    num_samples: int = 100
) -> pd.DataFrame:
    """
    Генерировать прогноз на реальных данных.
    """
    # Подготовка контекста
    context_data = df[feature_columns].iloc[-config.context_length:].values
    context_scaled = scaler.transform(context_data)
    context_tensor = torch.FloatTensor(context_scaled).unsqueeze(0).to(device)
    
    # Генерация прогнозов
    model.eval()
    with torch.no_grad():
        samples = model.sample(context_tensor, num_samples)
    
    samples_np = samples.cpu().numpy()[0]  # [num_samples, pred_len]
    
    # Денормализация (только для close price - индекс 0)
    close_mean = scaler.mean_[0]
    close_std = scaler.scale_[0]
    
    samples_denorm = samples_np * close_std + close_mean
    
    # Формируем DataFrame с прогнозами
    last_timestamp = df['timestamp'].iloc[-1]
    forecast_times = pd.date_range(
        start=last_timestamp + pd.Timedelta(hours=1),
        periods=config.prediction_length,
        freq='H'
    )
    
    forecast_df = pd.DataFrame({
        'timestamp': forecast_times,
        'mean': samples_denorm.mean(axis=0),
        'std': samples_denorm.std(axis=0),
        'q10': np.percentile(samples_denorm, 10, axis=0),
        'q50': np.percentile(samples_denorm, 50, axis=0),
        'q90': np.percentile(samples_denorm, 90, axis=0),
    })
    
    return forecast_df

# Генерируем прогноз
forecast_df = generate_real_forecast(
    model, df_features, feature_columns, scaler, config, num_samples=100
)

print("=== Прогноз на следующие 24 часа ===")
forecast_df

In [ ]:
# Визуализация прогноза
fig, ax = plt.subplots(figsize=(14, 6))

# Историческая цена (последние 7 дней)
history = df.tail(24 * 7)
ax.plot(history['timestamp'], history['close'], 'b-', linewidth=1, label='Historical')

# Прогноз
ax.fill_between(
    forecast_df['timestamp'],
    forecast_df['q10'],
    forecast_df['q90'],
    alpha=0.3,
    color='orange',
    label='80% CI'
)
ax.plot(
    forecast_df['timestamp'],
    forecast_df['mean'],
    'orange',
    linewidth=2,
    label='Forecast'
)

# Вертикальная линия - граница прогноза
ax.axvline(df['timestamp'].iloc[-1], color='gray', linestyle='--', alpha=0.5)

ax.set_xlabel('Date')
ax.set_ylabel('Price (USD)')
ax.set_title('TimeGrad BTC/USDT 24-Hour Forecast')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 10. Выводы

### Преимущества TimeGrad:

1. **Вероятностные прогнозы**: Модель генерирует распределение возможных исходов, а не точечную оценку
2. **Естественная оценка неопределённости**: Квантили позволяют строить доверительные интервалы
3. **Гибкость**: RNN-энкодер хорошо захватывает временные зависимости

### Ограничения:

1. **Вычислительная сложность**: Генерация сэмплов требует много итераций диффузии
2. **Авторегрессивность**: Ошибки накапливаются для длинных горизонтов
3. **Калибровка**: Может потребоваться дополнительная настройка для корректного покрытия

### Рекомендации:

- Используйте DDIM сэмплирование для ускорения генерации
- Рассмотрите ансамбль моделей для улучшения калибровки
- Для длинных горизонтов рассмотрите non-autoregressive методы (CSDI)

In [ ]:
# Сохранение модели
torch.save({
    'model_state_dict': model.state_dict(),
    'config': config,
    'scaler_mean': scaler.mean_,
    'scaler_scale': scaler.scale_,
    'feature_columns': feature_columns,
    'metrics': metrics
}, 'timegrad_btcusdt.pt')

print("Модель сохранена в timegrad_btcusdt.pt")